In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from sklearn.model_selection import GroupKFold , KFold

import os

print_line = 0
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print_line += 1
        if print_line < 5:
            print(os.path.join(dirname, filename))
        else:
            pass
        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output w|hen you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
if not iskaggle:
    import zipfile,kaggle
    path = Path('feedback-prize-effectiveness')
    kaggle.api.competition_download_cli(str(path))
    zipfile.ZipFile(f'{path}.zip').extractall(path)

In [ ]:
from fastai.imports import *

In [ ]:
if iskaggle: path = Path('../input/feedback-prize-effectiveness')
path.ls()

In [ ]:
def reduce_mem_usage(df,verbose=True):
    cols = df.columns.tolist()
    
    numerics = ['int8','int32','int16','int64','float16','float32','float64']
    memory = df.memory_usage().sum() / 1024 ** 2  
    
    
    for col in cols:
        col_type = df[col].dtypes
        if col_type in numerics:
            col_min = df[col].min()
            print(col_min)
            col_max = df[col].max()
            if str(col_type[:3]) == 'int': 
                if col_min > np.iinfo(np.int32).min() and  col_max > np.iinfo(np.int32).max():
                    df[col] =df[col].astype(np.int32)
                if col_min > np.iinfo(np.int64).min() and  col_max > np.iinfo(np.int64).max():
                    df[col] =df[col].astype(np.int64)
                if col_min > np.iinfo(np.int16).min() and  col_max > np.iinfo(np.int16).max():
                    df[col] =df[col].astype(np.int16)
                if col_min > np.iinfo(np.int8).min() and  col_max > np.iinfo(np.int8).max():
                    df[col] =df[col].astype(np.int8)
            else:
                if col_min > np.iinfo(np.float32).min() and  col_max > np.iinfo(np.float32).max():
                    df[col] = df[col].astype(np.float32)
                if col_min > np.iinfo(np.float64).min() and  col_max > np.iinfo(np.float64).max():
                    df[col] = df[col].astype(np.float64) 
    if verbose: print('memory reduce : {}'.format(memory - df.memory_usage().sum() /1024 **2))
    return df

In [ ]:

train = pd.read_csv('../input/feedback-prize-effectiveness/train.csv')

In [ ]:
test  = pd.read_csv(path/'test.csv')

In [ ]:
len(train.discourse_id.unique()) == len(train)

In [ ]:
train.essay_id.value_counts()

In [ ]:
train.loc[train.essay_id == '91B1F82B2CF1'].head()

In [ ]:
train['discourse_text']

In [ ]:
essay_len = []
for file in os.listdir(path/'train'):
    with open (path/'train'/file) as f:
        essay_len.append(len(f.read()))

In [ ]:
min(essay_len) , np.mean(essay_len) , max(essay_len)

In [ ]:
plt.hist(essay_len)

In [ ]:
text_len = [len(text) for text in train.discourse_text]

In [ ]:
import matplotlib.pyplot as plt
plt.hist(text_len)

In [ ]:
train.discourse_type.value_counts()

In [ ]:
import warnings,transformers,logging,torch
from transformers import TrainingArguments,Trainer
from transformers import AutoModelForSequenceClassification,AutoTokenizer

In [ ]:
if iskaggle:
    !pip install --no-index --find-links ../input/huggingface-datasets datasets -q
import datasets
from datasets import load_dataset, Dataset, DatasetDict

In [ ]:
warnings.simplefilter('ignore')
logging.disable(logging.WARNING)


In [ ]:
model_nm = '../input/debertav3small'

In [ ]:
sep

In [ ]:
df.discourse_type + sep + df.discourse_text

In [ ]:
df['inputs'] = df.discourse_type + sep +df.discourse_text

In [ ]:
new_label = {"discourse_effectiveness" : {"Ineffective" : 0, "Adequate" : 1, "Effective" :2}}
train = train.replace(new_label)

In [ ]:
train.rename(columns = {"discourse_effectiveness" : "label"})d\

In [ ]:
ds = Dataset.from_pandas(train)

In [ ]:
def tok_func(x): return tokz(x["inputs"], truncation = True)

In [ ]:
def tok_func(x): return tokz(x["inputs"], truncation = True)

In [ ]:
tokz = AutoTokenizer.from_pretrained(model_nm)
sep = tokz.sep_token
train['inputs'] = train.discourse_type + sep + train.discourse_text
#We can now tokenize the the input. We'll use Dataset.map to speed it up, and remove the columns we no longer need:
tok_ds = ds.map(tok_func, batched=True , remove_columns= inps+('inputs','discourse_id','essay_id'))

In [ ]:
ds[0]['inputs']

In [ ]:
inps = "discourse_text" , "discourse_type"

In [ ]:
#We can now tokenize the the input. We'll use Dataset.map to speed it up, and remove the columns we no longer need:
tok_ds = ds.map(tok_func, batched=True , remove_columns= inps+('inputs','discourse_id','essay_id'))

In [ ]:
tok_ds[0].keys()

In [ ]:
essay_ids = train.essay_id.unique()

In [ ]:
np.random.seed(42)
np.random.shuffle(essay_ids)

In [ ]:
essay_ids[:5]

In [ ]:
val_prop = 0.2
val_sz = int(len(essay_ids)*val_prop)
val_essay_ids  = essay_ids[:val_sz]

In [ ]:
is_val = np.isin(train.essay_id, val_essay_ids)

In [ ]:
idxs = np.arange(len(train))

In [ ]:
idxs

In [ ]:
val_idxs = idxs[is_val]

In [ ]:
trn_idxs  =idxs[~is_val]

In [ ]:
val_idxs

In [ ]:
len(val_idxs), len(trn_idxs)

In [ ]:
tok_ds

In [ ]:
dds = DatasetDict({"train" : tok_ds.select(trn_idxs), 
                  "test" : tok_ds.select(val_idxs)})

In [ ]:
def get_dds(df, train=True):
    ds = Dataset.from_pandas(df)
    to_remove = ['discourse_text','discourse_type', 'inputs', 'discourse_id','essay_id']
    tok_ds  = ds.map(tok_func , batched=True, remove_columns = to_remove)

    if train:
        return DatasetDict({"train" : tok_ds.select(trn_idxs), "test" : tok_ds.select(val_idxs)})
    else:
        return tok_ds

In [ ]:
ds = Dataset.from_pandas(df)

In [ ]:
tok_ds['input_ids']

In [ ]:

ds.map(tok_func)

In [ ]:
tok_ds

In [ ]:
lr , bs = 8e-5, 16
wd, epochs = 0.01, 1

In [ ]:
from sklearn.metrics import log_loss
import torch.nn.functional as F
def score(preds): return {'log_loss' : log_loss(preds.label_ids, F.softmax(torch.Tensor(preds.predictions)))}

In [ ]:
def get_trainer(dds):
    args = TrainingArguments('outpus' , learning_rate = lr, warmup_ratio=0.1, 
                    lr_scheduler_type='cosine' , fp16= True ,evaluation_strategy="epoch", per_device_train_batch_size=bs ,
                            per_device_eval_batch_size = bs*2, num_train_epochs = epochs, 
                             weight_decay = wd, report_to ='none')
    model = AutoModelForSequenceClassification.from_pretrained(model_nm,num_labels=3)
    return Trainer(model , args, train_dataset=dds['train'], eval_dataset=dds['test'] ,
                  tokenizer=tokz, compute_metrics=score )

In [ ]:
trainer = get_trainer(dds)

In [ ]:
trainer.train()

In [ ]:
# fp16=True error


In [ ]:
test_df = 1

In [ ]:
def get_trainer(dds):
    args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
        evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
        num_train_epochs=epochs, weight_decay=wd, report_to='none')
    model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=3)
    return Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                   tokenizer=tokz, compute_metrics=score)

In [ ]:
def get_trainer(dds):
    args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
        evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
        num_train_epochs=epochs, weight_decay=wd, report_to='none')
    model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=3)
    return Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                   tokenizer=tokz, compute_metrics=score)

In [ ]:
trainer = get_trainer(dds)

In [ ]:
trainer.train()

In [ ]:
# 이전대회 솔루션

# https://www.kaggle.com/competitions/feedback-prize-2021/code?competitionId=31779&sortBy=voteCount


In [ ]:
#ds[0]['inputs']

In [ ]:
#tok_func(ds[0])

In [1]:
from pathlib import Path
import os

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
if not iskaggle:
    import zipfile,kaggle
    path = Path('feedback-prize-effectiveness')
    kaggle.api.competition_download_cli(str(path))
    zipfile.ZipFile(f'{path}.zip').extractall(path)

In [2]:
from fastai.imports import *

In [3]:
if iskaggle: path = Path('../input/feedback-prize-effectiveness')
path.ls()

(#5) [Path('../input/feedback-prize-effectiveness/sample_submission.csv'),Path('../input/feedback-prize-effectiveness/train.csv'),Path('../input/feedback-prize-effectiveness/test.csv'),Path('../input/feedback-prize-effectiveness/test'),Path('../input/feedback-prize-effectiveness/train')]

In [87]:
df = pd.read_csv(path/'train.csv')
df.head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about how this face on Mars is a natural landform or if there is life on Mars that made it. The story is about how NASA took a picture of Mars and a face was seen on the planet. NASA doesn't know if the landform was created by life on Mars, or if it is just a natural landform.",Lead,Adequate
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a natural landform because I dont think that there is any life on Mars. In these next few paragraphs, I'll be talking about how I think that is is a natural landform",Position,Adequate
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform because there is no life on Mars that we have descovered yet,Claim,Adequate
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The reason why I think it is a natural landform because, nobody live on Mars in order to create the figure. It says in paragraph 9, ""It's not easy to target Cydonia,"" in which he is saying that its not easy to know if it is a natural landform at this point. In all that they're saying, its probably a natural landform.",Evidence,Adequate
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by alieans because they thought that there was life on Mars.,Counterclaim,Adequate


In [5]:
test_df = pd.read_csv(path/'test.csv')
test_df.head()

,discourse_id,essay_id,discourse_text,discourse_type
0,a261b6e14276,D72CB1C11673,Making choices in life can be very difficult. People often ask for advice when they can not decide on one thing. It's always good to ask others for their advice when making a choice. When you have multiple opinions you have the ability to make the best choice for yourself.,Lead
1,5a88900e7dc1,D72CB1C11673,Seeking multiple opinions can help a person make a better choice,Position
2,9790d835736b,D72CB1C11673,it can decrease stress levels,Claim
3,75ce6d68b67b,D72CB1C11673,a great chance to learn something new,Claim
4,93578d946723,D72CB1C11673,can be very helpful and beneficial.,Claim


In [6]:
essay_len = []
for file in os.listdir(path/'train'):
    with open(path/'train'/file) as f:
        essay_len.append(len(f.read()))

In [7]:
text_len = [len(x) for x in df.discourse_text]

In [8]:
sample_df = pd.read_csv(path/'sample_submission.csv')
sample_df.head()

,discourse_id,Ineffective,Adequate,Effective
0,a261b6e14276,0.20,0.60,0.40
1,5a88900e7dc1,3.00,6.00,1.00
2,9790d835736b,1.00,2.00,3.00
3,75ce6d68b67b,0.33,0.34,0.33
4,93578d946723,0.01,0.24,0.47


In [9]:
from torch.utils.data import DataLoader
import warnings,transformers,logging,torch
from transformers import TrainingArguments,Trainer
from transformers import AutoModelForSequenceClassification,AutoTokenizer

In [10]:
if iskaggle:
    !pip install --no-index --find-links ../input/huggingface-datasets datasets -q
import datasets
from datasets import load_dataset, Dataset, DatasetDict

In [11]:
warnings.simplefilter('ignore')
logging.disable(logging.WARNING)

In [12]:
model_nm = '../input/debertav3small'

In [13]:
tokz = AutoTokenizer.from_pretrained(model_nm)

In [14]:
sep = tokz.sep_token
sep

'[SEP]'

In [15]:
df['inputs'] = df.discourse_type + sep +df.discourse_text

In [16]:
new_label = {"discourse_effectiveness": {"Ineffective": 0, "Adequate": 1, "Effective": 2}}
df = df.replace(new_label)
df = df.rename(columns = {"discourse_effectiveness": "label"})

In [17]:
ds = Dataset.from_pandas(df)

In [18]:
def tok_func(x): return tokz(x["inputs"], truncation=True)

In [19]:
inps = "discourse_text","discourse_type"
tok_ds = ds.map(tok_func, batched=True, remove_columns=inps+('inputs','discourse_id','essay_id'))

  0%|          | 0/37 [00:00<?, ?ba/s]

In [20]:
tok_ds

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 36765
})

In [33]:
from transformers import BertTokenizer
tz = BertTokenizer.from_pretrained("bert-base-cased")
tz.convert_tokens_to_ids(["characteristically"])

sent = "He remains characteristically confident and optimistic."
tz.tokenize(sent)
['He',
 'remains',
 'characteristic',
 '##ally',
 'confident',
 'and',
 'optimistic',
 '.']

tz.convert_tokens_to_ids(tz.tokenize(sent))

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

[1124, 2606, 7987, 2716, 9588, 1105, 24876, 119]

In [88]:
df['discourse_text_tk'] = df.discourse_text.map(tz.tokenize)

In [90]:
df['discourse_text_tk_ids'] = df['discourse_text_tk'].map(tz.convert_tokens_to_ids)

In [96]:
df['token_pad'] = df.discourse_text_tk_ids.map(token_pad)

In [98]:
df = df.rename(columns = {"discourse_effectiveness": "label"})

In [ ]:
def token_pad(x):
    return ([1] * len(x)) +( [0] * (max_len-len(x)))

In [105]:
df['discourse_text_tk_ids'] = df['discourse_text_tk_ids'].map(lambda x : x + ([0] * (max_len-len(x))))

In [107]:
 
from sklearn.preprocessing import LabelEncoder

In [110]:
encoder = LabelEncoder()

In [113]:
df['label'] = encoder.fit_transform(df['label'])

In [115]:
from torch.utils.data import DataLoader

train_loaderDataLoader(CateDataset , batch_size )

NameError: name 'train_loaderDataLoader' is not defined

In [ ]:
train_res = train(train_loader)

In [ ]:
model = cat_e_model.CateClassifier(CFG)


In [ ]:
def train(train_loader, model, optimizer

In [106]:
# token_types (segment)

,discourse_id,essay_id,discourse_text,discourse_type,label,discourse_text_tk,discourse_text_tk_ids,token_pad
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about how this face on Mars is a natural landform or if there is life on Mars that made it. The story is about how NASA took a picture of Mars and a face was seen on the planet. NASA doesn't know if the landform was created by life on Mars, or if it is just a natural landform.",Lead,Adequate,"[Hi, ,, i, ', m, Isaac, ,, i, ', m, going, to, be, writing, about, how, this, face, on, Mars, is, a, natural, land, ##form, or, if, there, is, life, on, Mars, that, made, it, ., The, story, is, about, how, NASA, took, a, picture, of, Mars, and, a, face, was, seen, on, the, planet, ., NASA, doesn, ', t, know, if, the, land, ##form, was, created, by, life, on, Mars, ,, or, if, it, is, just, a, natural, land, ##form, .]","[8790, 117, 178, 112, 182, 7026, 117, 178, 112, 182, 1280, 1106, 1129, 2269, 1164, 1293, 1142, 1339, 1113, 7403, 1110, 170, 2379, 1657, 13199, 1137, 1191, 1175, 1110, 1297, 1113, 7403, 1115, 1189, 1122, 119, 1109, 1642, 1110, 1164, 1293, 9085, 1261, 170, 3439, 1104, 7403, 1105, 170, 1339, 1108, 1562, 1113, 1103, 5015, 119, 9085, 2144, 112, 189, 1221, 1191, 1103, 1657, 13199, 1108, 1687, 1118, 1297, 1113, 7403, 117, 1137, 1191, 1122, 1110, 1198, 170, 2379, 1657, 13199, 119, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a natural landform because I dont think that there is any life on Mars. In these next few paragraphs, I'll be talking about how I think that is is a natural landform",Position,Adequate,"[On, my, perspective, ,, I, think, that, the, face, is, a, natural, land, ##form, because, I, don, ##t, think, that, there, is, any, life, on, Mars, ., In, these, next, few, paragraph, ##s, ,, I, ', ll, be, talking, about, how, I, think, that, is, is, a, natural, land, ##form]","[1212, 1139, 7281, 117, 146, 1341, 1115, 1103, 1339, 1110, 170, 2379, 1657, 13199, 1272, 146, 1274, 1204, 1341, 1115, 1175, 1110, 1251, 1297, 1113, 7403, 119, 1130, 1292, 1397, 1374, 24950, 1116, 117, 146, 112, 1325, 1129, 2520, 1164, 1293, 146, 1341, 1115, 1110, 1110, 170, 2379, 1657, 13199, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform because there is no life on Mars that we have descovered yet,Claim,Adequate,"[I, think, that, the, face, is, a, natural, land, ##form, because, there, is, no, life, on, Mars, that, we, have, des, ##cover, ##ed, yet]","[146, 1341, 1115, 1103, 1339, 1110, 170, 2379, 1657, 13199, 1272, 1175, 1110, 1185, 1297, 1113, 7403, 1115, 1195, 1138, 3532, 17800, 1174, 1870, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [104]:
df

,discourse_id,essay_id,discourse_text,discourse_type,label,discourse_text_tk,discourse_text_tk_ids,token_pad
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about how this face on Mars is a natural landform or if there is life on Mars that made it. The story is about how NASA took a picture of Mars and a face was seen on the planet. NASA doesn't know if the landform was created by life on Mars, or if it is just a natural landform.",Lead,Adequate,"[Hi, ,, i, ', m, Isaac, ,, i, ', m, going, to, be, writing, about, how, this, face, on, Mars, is, a, natural, land, ##form, or, if, there, is, life, on, Mars, that, made, it, ., The, story, is, about, how, NASA, took, a, picture, of, Mars, and, a, face, was, seen, on, the, planet, ., NASA, doesn, ', t, know, if, the, land, ##form, was, created, by, life, on, Mars, ,, or, if, it, is, just, a, natural, land, ##form, .]","[8790, 117, 178, 112, 182, 7026, 117, 178, 112, 182, 1280, 1106, 1129, 2269, 1164, 1293, 1142, 1339, 1113, 7403, 1110, 170, 2379, 1657, 13199, 1137, 1191, 1175, 1110, 1297, 1113, 7403, 1115, 1189, 1122, 119, 1109, 1642, 1110, 1164, 1293, 9085, 1261, 170, 3439, 1104, 7403, 1105, 170, 1339, 1108, 1562, 1113, 1103, 5015, 119, 9085, 2144, 112, 189, 1221, 1191, 1103, 1657, 13199, 1108, 1687, 1118, 1297, 1113, 7403, 117, 1137, 1191, 1122, 1110, 1198, 170, 2379, 1657, 13199, 119]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a natural landform because I dont think that there is any life on Mars. In these next few paragraphs, I'll be talking about how I think that is is a natural landform",Position,Adequate,"[On, my, perspective, ,, I, think, that, the, face, is, a, natural, land, ##form, because, I, don, ##t, think, that, there, is, any, life, on, Mars, ., In, these, next, few, paragraph, ##s, ,, I, ', ll, be, talking, about, how, I, think, that, is, is, a, natural, land, ##form]","[1212, 1139, 7281, 117, 146, 1341, 1115, 1103, 1339, 1110, 170, 2379, 1657, 13199, 1272, 146, 1274, 1204, 1341, 1115, 1175, 1110, 1251, 1297, 1113, 7403, 119, 1130, 1292, 1397, 1374, 24950, 1116, 117, 146, 112, 1325, 1129, 2520, 1164, 1293, 146, 1341, 1115, 1110, 1110, 170, 2379, 1657, 13199]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform because there is no life on Mars that we have descovered yet,Claim,Adequate,"[I, think, that, the, face, is, a, natural, land, ##form, because, there, is, no, life, on, Mars, that, we, have, des, ##cover, ##ed, yet]","[146, 1341, 1115, 1103, 1339, 1110, 170, 2379, 1657, 13199, 1272, 1175, 1110, 1185, 1297, 1113, 7403, 1115, 1195, 1138, 3532, 17800, 1174, 1870]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The reason why I think it is a natural landform because, nobody live on Mars in order to create the figure. It says in paragraph 9, ""It's not easy to target Cydonia,"" in which he is saying that its not easy to know if it is a natural landform at this point. In all that they're saying, its probably a natural landform.",Evidence,Adequate,"[If, life, was, on, 

In [91]:
max_len = max([len(token_ids) for token_ids in df.discourse_text_tk.values.tolist()])

In [82]:
df['discourse_text_tk'].values[0]

array([list([8790, 117, 178, 112, 182, 7026, 117, 178, 112, 182, 1280, 1106, 1129, 2269, 1164, 1293, 1142, 1339, 1113, 7403, 1110, 170, 2379, 1657, 13199, 1137, 1191, 1175, 1110, 1297, 1113, 7403, 1115, 1189, 1122, 119, 1109, 1642, 1110, 1164, 1293, 9085, 1261, 170, 3439, 1104, 7403, 1105, 170, 1339, 1108, 1562, 1113, 1103, 5015, 119, 9085, 2144, 112, 189, 1221, 1191, 1103, 1657, 13199, 1108, 1687, 1118, 1297, 1113, 7403, 117, 1137, 1191, 1122, 1110, 1198, 170, 2379, 1657, 13199, 119]),
       list(['Hi', ',', 'i', "'", 'm', 'Isaac', ',', 'i', "'", 'm', 'going', 'to', 'be', 'writing', 'about', 'how', 'this', 'face', 'on', 'Mars', 'is', 'a', 'natural', 'land', '##form', 'or', 'if', 'there', 'is', 'life', 'on', 'Mars', 'that', 'made', 'it', '.', 'The', 'story', 'is', 'about', 'how', 'NASA', 'took', 'a', 'picture', 'of', 'Mars', 'and', 'a', 'face', 'was', 'seen', 'on', 'the', 'planet', '.', 'NASA', 'doesn', "'", 't', 'know', 'if', 'the', 'land', '##form', 'was', 'created', 'by', 'life', '

In [ ]:
df['token_mask'].map(lambda x : ([1] * len(x)) +( [0] * (max_len-len(x)))
                     
                                      

In [21]:
essay_ids = df.essay_id.unique()
np.random.seed(42)
np.random.shuffle(essay_ids)
essay_ids[:5]

array(['B5C606F0A883', 'FA4FE7706A1A', '37A77BEAD718', '0ED28D8A5EC4',
       'F25BA634ADDD'], dtype=object)

In [22]:
val_prop = 0.2
val_sz = int(len(essay_ids)*val_prop)
val_essay_ids = essay_ids[:val_sz]

In [23]:
is_val = np.isin(df.essay_id, val_essay_ids)
idxs = np.arange(len(df))
val_idxs = idxs[ is_val]
trn_idxs = idxs[~is_val]
len(val_idxs),len(trn_idxs)

(7181, 29584)

In [24]:
dds = DatasetDict({"train":tok_ds.select(trn_idxs),
             "test": tok_ds.select(val_idxs)})

In [25]:
def get_dds(df, train=True):
    ds = Dataset.from_pandas(df)
    to_remove = ['discourse_text','discourse_type','inputs','discourse_id','essay_id']
    tok_ds = ds.map(tok_func, batched=True, remove_columns=to_remove)
    if train:
        return DatasetDict({"train":tok_ds.select(trn_idxs), "test": tok_ds.select(val_idxs)})
    else: 
        return tok_ds

In [26]:
lr,bs = 8e-5,16
wd,epochs = 0.01,1

In [27]:
from sklearn.metrics import log_loss
import torch.nn.functional as F
def score(preds): return {'log loss': log_loss(preds.label_ids, F.softmax(torch.Tensor(preds.predictions)))}

In [28]:
def get_trainer(dds):
    args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
        evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
        num_train_epochs=epochs, weight_decay=wd, report_to='none')
    model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=3)
    return Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                   tokenizer=tokz, compute_metrics=score)

In [30]:
train

NameError: name 'train' is not defined

In [29]:
trainer = get_trainer(dds)
trainer.train()

ValueError: Mixed precision training with AMP or APEX (`--fp16` or `--bf16`) and half precision evaluation (`--fp16_full_eval` or `--bf16_full_eval`) can only be used on CUDA devices.

In [ ]:
test_df = pd.read_csv(path/'test.csv')

In [ ]:
test_df['inputs'] = test_df.discourse_type + sep + test_df.discourse_text

In [ ]:
test_ds  = get_dds(test_df, train=False)

In [ ]:
preds = F.softmax(torch.Tensor(trainer.predict(test_ds).predictions)).numpy().astype(float)
preds

In [ ]:
F.softmax(torch.Tensor(trainer.predict(test_ds).predictions)).numpy().astype(float)

In [ ]:
preds

In [ ]:
import torch
import torch.nn as nn

In [ ]:

from transformers import BertConfig, BertModel

In [ ]:
df

In [ ]:
class CateClassifier(nn.Module):
    def __init__(self, cfg):
        """
        매개변수
        cfg : hidden_size , nlayers 등 설정값을 가지고 있는 변수
        """
        
        super(CateClassifier, self).__init__()
        # 글로벌 설정값을 멤버 변수로 저장
        self.cfg = cfg
        # 버트모델의 설정값을 멤버 변수로 저장
        self.bert_cfg = BertConfig(
            cfg.vocab_size, # 사전 크기
            hidden_size = cfg.hidden_size, # 히든 크기 
            num_layers=cfg.nlayers, # 레이어 층 수 
            num_attention_heads=cfg.nheads, # 어텐션 헤드의 수 
            intermediate_size=cfg.intermediate_size,  # 인터미디어트 크기
            hidden_dropout_prob=cfg.dropout, # 히든 드롭아웃 확률 값
            attention_probs_dropout_prob=cfg.dropout, # 어텐션 드롭아웃 확률 값
            max_position_embeddings=cfg.seq_len, # 포지션 임베딩의 최대 길이
            type_vocab_size=cfg.type_vocab_size , # 타입 사전 크기
        )
        # 텍스트 인코더로 버트모델 사용
        self.text_encoder = BertModel(self.bert_cfg)
        # 이미지 인코더로 선형모델 사용( 대회에서 이미지가 아닌 img_feat를 제공)
        self.img_encoder=  nn.Linear(cfg.img_feat_size, cfg.hidden_size)
        
        # 분류기(Classifier) 생성기
        def get_cls(target_size=1):
            return nn.Sequential(
                nn.Linear(cfg.hidden_size*2, cfg.hidden_size),
                nn.LayerNorm(cfg.hidden_size),
                nn.Dropout(cfg.dropout),
                nn.ReLU(),
                nn.Linear(cfg.hidden_size, target_size)
            )
        
        # 대 카테고리 분류기
        self.b_cls  = get_cls(cfg.n_b_cls)
        
        
        def forward(self, token_ids , token_mask , token_types, img_feat , label=None):
            """
            매개변수
            token_ids : 전처리된 상품명을 인덱스로 변환하여 token_ids를 만들었음
            token_mask : 실제 token_ids 의 개수만큼은 1, 나머지는 0 
            token_types: _ 문자를 기준으로 서로 다른 타입의 토큰임을 타입 인덱스로 저장
            img_feat: resnet50으로 인코딩된 이미지 피처
            label: 정답 대/중/소/세 카테고리
            """
            
            # 전처리된 상품명을 하나의 텍스트벡터(text_vec)로 변환
            # 반환 튜플(시퀀스 아웃풋, 풀드(pooled) 아웃풋) 중 시퀀스 아웃풋만 참조
            text_output = self.text_encoder(token_ids, token_mask,
                                           token_type_ids= token_types)[0]
            
            # 시퀀스 중 첫 타입스텝의 hidden state만 사용
            text_vec = text_output[:,0]
            
            # img_feat를 텍스트벡터와 결합하기 앞서 선형변환 적용
            img_vec = self.img_encoder(img_feat)
            
            # 이미지벡터와 텍스트벡터를 직렬연결(concatenate)하여 결합벡터 생성
            comb_vec = torch.cat([text_vec, img_vec],1 )
            
            # 결합된 벡터로 대카테고리 확률분포 예측
            b_pred = self.b_cls(comb_vec)
            
            
            # 데이터 페럴렐 학습에서 gpu 메모리를 효율적으로 사용하기 위해 
            if label is not None:
                # 손실(loss) 함수로 CrossEntropyLoss를 사용
                loss_func = nn.CrossEntropyLoss(ignore_index)
                
                b_label, m_label, s_label, d_label = label.split(1,1)
                
                b_loss = loss_func(b_pred, b_label.view(-1))
                
                m_loss =loss_func(m_pred , m_label.view(-1))
                
                s_loss = loss_func(s_pred, s_label.view(-1))
                
                d_loss = loss_func(d_pred, d_label.view(-1))
                
                loss = (b_loss+ 1.2*m_loss + 1.3*s_loss+ 1.4*d_loss)/4
            else:
                loss = b_pred.new(1).fill_(0)
                
                
            return loss, [b_pred, m_pred, s_pred, d_pred]
        

In [ ]:
import torch
from torch.utils.data import Dataset
import h5py
import re

class CateDataset(Dataset):
    """
    데이터셋에서 학습에 필요한 형태로 변환된 샘플 하나를 반환
    """
    def __init__(self, df_data, img_h5_path , token2id, tokens_max_len=64,
                type_vocab_size=30):
        """
        매개변수
        df_data:  상품타이틀 , 카테고리 등의 정보를 가지는 데이터프레임
        img_h5_path : img_feat가 저장되 있는 h5 파일의 경로
        token2id : token을 token_id 로 변환하기 위한 맵핑 정보를 가진 딕셔너리
        tokens_max_len : tokens 의 최대 길이. 상품명의 tokens 가 이 이상이면 잘라서 버림 
        type_vocab_size : 타입 사전의 크기
        """
        self.tokens = df_data['tokens'].values # 전처리된 상품명
        self.img_indices = df_data['img_idx'].values # h5의 이미지 인덱스
        self.img_h5_path = img_h5_path
        self.tokens_max_len = tokens_max_len
        self.labels = df_data[['bcateid','mcateid', 'scateid', 'dcateid']].values
        self.token2id = token2id
        self.p = re.compile('_[^_]+') # _ 기호를 기준으로 나누기 위한 컴파일된 정규식 
        self.type_vocab_size = type_vocab_size
        
        
    def __getitem__(self, idx):
        """
        데이터셋에서 idx에 대응되는 샘플을 변환하여 반환
        """
        
        if idx >= len(self):
            raise StopIteration
            
        # idx 에 해당하는 상품명 가졍괴 . 상품명은 문자열로 저장돼 있음 
        tokens = self.tokens[idx]
        
        if not isinstance(tokens, str):
            tokens= ''
        
        
        # 상품명을 _기호를 기준으로 분리하여 파이썬 리스트로 저장
        # "_직소퍼즐 _1000 조각 _바다 거북 의 _여행 _pl 12 75" =>
        # ["_직소퍼즐" , "_1000 조각", "_바다 거북 의" , "_여행" , "_pl 12 75"]
        
        tokens = self.p.findall(tokens)
        
        
        # _ 기호별 토큰. 타입 인덱스 부여
        # ["_직소퍼즐" , "_1000 조각" ,"_바다 거북 의", "_여행", "_pl 12 75"]
        # [      0  ,       1   1         ,  2  2    2 , 3 , 4 4 4]
        token_types  = [type_id for type_id , word in enumerate(tokens) for _ in word.split()]
        
        tokens = " ".join(tokens) # _ 기호로 분리되기 전의 원래의 tokens로 되돌림
        
        # 토큰을 토큰에 대응되는 인덱스로 변환
        # "_직소퍼즐 _1000 조각 _바다 거북 의 _여행 _pl 12 75"  => 
        # [2291, 784, 2179 , 3540 , 17334 , 30827 , 1114, 282 ,163, 444]
        # "_직소퍼즐" => 2291
        # "_1000" => 784
        # "조각" => 2179
        # ...
        token_ids = [self.token2id[tok] if tok in self.token2id else 0 for tok in tokens.split()]
        
        # token_ids의 길이가 max_len 보다 길면 잘라서 버림
        if len(token_ids) > self.tokens_max_len:
            token_ids = token_ids[:self.tokens_max_len]
            token_types  = token_types[:self.tokens_max_len]
        
        
        # token_ids의 길이가 max_len 보다 짧으면 짧은 만큼 PAD값을 0으로 채워 넣음 
        # token_ids 중 값이 있는 곳은 1, 그 외는 0으로 채운 token_mask 생성
        token_mask = [1] * len(token_ids)
        token_pad = [0] * (self.tokens_max_len - len(token_ids))
        token_ids += token_pad
    
        token_mask += token_pad
        token_types += token_pad # max_len 보다 짧은 만큼 PAD 추가
        
        # h5파일에서 이미지 인덱스에 해당하는 img_feat를 가져옴
        # 파이토치의 데이터 로더에 의해 h5 파일에 동시접근이 발생해도
        # 안정적으로 img_feat를 가져오려면 아래처럼 매번 h5py.File 호출 필요
        with h5py.File(self.img_h5_path,'r') as img_feats:
            img_feat = img_feats['img_feat'][self.img_indices[idx]]
        
        
        # 넘파이(numpy) 나 파이썬 자료형을 파이토치의 자료형으로 변환
        token_ids = torch.LongTensor(token_ids)
        token_mask = torch.LongTensor(token_mask)
        token_types = torch.LongTensor(token_types)
        
        #token_types 의 타입 인덱스의 숫자 크기가 type_vocab_size보다 작도록 바꿈
        token_types[toekn_types >= self.type_vocab_size] = self.type_vocab_size -1 
        img_feat = torch.FloatTensor(img_feat)
        
        # 대/중/소/세 레이블 준비
        label  =self.labels[idx]
        label = torch.LongTensor(label)
        
        # 크게 3가지 텍스트 입력 , 이미지 입력 , 레이블을 반환함
        return token_ids , token_mask , token_types , img_feat, label
    
    
    def __len__(self):
        """
        tokens의 개수를 반환한다. 즉 , 상품명 문장의 개수를 반환한다
        
        """
        return len(self.tokens)
    
    
        # ["_직소퍼즐","_1000 조각", "_바다 거북 의", "_여행", "_pl 12 75"]
        tokens = self.p.findall(tokens)
        # _ 기호별 토큰타입 인덱스 부여
        # ["_직소퍼즐" , "_1000 조각" , "_바다 거북 의", "_여행" , "_pl 12 75"] => 
        # [    0       ,     1   1 ,     2       2  2 ,     3 , 4 4   4   ]
        token_types = [type_id for type_id , word in enumerate(tokens) for _ in word.split()]
        
        

In [ ]:
tokens = 

In [ ]:
p.findall(tokens)

In [ ]:
tokz = AutoTokenizer.from_pretrained(model_nm)
sep = tokz.sep_token
train['inputs'] = train.discourse_type + sep + train.discourse_text
#We can now tokenize the the input. We'll use Dataset.map to speed it up, and remove the columns we no longer need:
tok_ds = ds.map(tok_func, batched=True , remove_columns= inps+('inputs','discourse_id','essay_id'))

In [ ]:
from transformers import BertConfig

In [ ]:
bert_cfg = BertConfig(
        vocab_size, 
    hidden_size = 500,
    num_hidden_layers,
    num_attention_head,
    intermediate_size,
    hidden_dropout_prob,
    attention_probs_dropout_prob,
    max_position_embeddings,
    type_vocab_size
)

In [ ]:
class CFG:
    batch_size = 1024
    num_workers = 4
    print_freq = 100
    warmup_steps  =100 # lr을 서서히 증가시킬 step 수
    hidden_size = 512 # 은닉 크기
    dropout=0.2 # dropout 확률
    intermediate_size = 256 # TRANSFORMER셀의 intermediate 크기
    nlayers =2 # BERT의 층 수
    nheads = 8 # BERT의 head 개수
    seq_len = 64 # 토큰의 최대 길이
    n_b_cls = 57 + 1 # 대카테고리 개수 
    n_m_cls = 552 + 1 # 중카테고리 개수 
    n_s_cls = 3190 + 1 # 소카테고리 개수 
    n_d_cls = 404 + 1 # 세카테고리 개수
    vocab_size = 32000  #토큰의 유니크 인덱스 개수 
    img_feat_size = 2048 # 이미지 피처 벡터의 크기
    type_vocab_size = 30 # 타입의 유니크 인덱스 개수
    #csv_path = os.path.join(DB_DIR, 'dev.csv') # 전처리 돼 저장된 dev 데이터셋
    #h5_path = os.path.join(DB_DIR , 'dev_img_feat.h5')

In [ ]:
cfg = CFG

In [ ]:
cfg.vocab_size

In [ ]:
cfg.dropout

In [ ]:
bert_cfg = BertConfig(
    cfg.vocab_size,
    hidden_size = cfg.hidden_size,
    num_hidden_layers = cfg.nlayers, # 레이어 층 수
    num_attention_heads=cfg.nheads, # 어텐션 헤드의 수 
    intermediate_size=  cfg.intermediate_size, # 인터미디어트 크기
    hidden_dropout_prob=cfg.dropout, # 히든 드롭아웃 확률 값
    attention_probs_dropout_prob=cfg.dropout, # 어텐션 드롭앙수 확률 값
    max_position_embeddings= cfg.seq_len, # 포지션 임베딩의 최대 길이
    type_vocab_size = cfg.type_vocab_size , # 타입 사전 크기
)

In [ ]:
text_encoder = BertModel(bert_cfg)

In [ ]:
text_encoder(token_ids, token_mask, token_type_ids= token_types)[0]

In [ ]:
dev_loader = DataLoader( dev_db, batch_size = CFG.batch_size, shuffle=False,
                       num_workers=CFG.num_workers, pin_memory=True)

In [ ]:
BertConfig(cfg.vocab_size)

In [ ]:
text_encoder = BertModel(bert_cfg)

In [ ]:
# 전처리된 데이터가 
DB_DIR = '../input/processed'

In [ ]:
DB_DIR = '../input/processed'

In [ ]:
h5_path = os.path.join(DB_DIR, 'dev_img_feat.h5')

In [ ]:
# 전처리돼서 저장된 dev.csv 를 불러오기
csv_path = os.path.join(DB_DIR, 'dev.csv')

In [ ]:
# 전처리 데이터 읽기
print('loading ...')
dev_df = pd.read_csv(CFG.csv_path ,dtype ={'tokens' : str})

In [ ]:
class CateDataset(Dataset):
    def __init__(self, df_data,img_h5_path, token2id, tokens_max_len=64, type_vocab_size=30):
        
        self.tokens = df_data['tokens'].values

In [ ]:
import sentencepiece as spm

In [ ]:
sp = spm.SentencePieceProcessor()

In [ ]:
csv_path = os.path.join(DB_DIR, 'dev.csv') 

In [ ]:
dev_df['tokens'] = dev_df['product'].map(lambda x: "".join([str(token_id) for token_id in sp.EncodeAsPieces(x)]))
dev_df.to_csv(os.path.join(PROCESSED_DATA_DIR, 'dev.csv'), index=False)
dev_df = pd.read_csv(CFG.csv_path, dtype={'tokens':str})    

In [ ]:
dev_db = cate_dataset.CateDataset(dev_df, img_h5_path, token2id , CFG.seq_len, CFG.type_vocab_size)

In [ ]:
# 다르게 전처리해서 어떻게 처리할 것인가?


In [ ]:
dev_loader = DataLoader(
    dev_db
)

inference(dev_loader, model_list)